The purpose of this note book is to apply the profit optiisation model to make predictions based on a set of met office data. This can then also be compared to Manon's results.

The code bellow imports the necessary python packages.

In [1]:
import sys
sys.path.append('../')

from src.vapour_pressure_deficit_calculation import vapour_pressure_deficit

from xarray import open_dataset
from pandas import DataFrame, date_range

# Metoffice data

To run the model we need a set of environmental data, this we get from a netcdf file. The code bellow imports and prints a description of this file.

In [2]:
metoffice_data_file_address = "../../Example met data/Alice_Holt/data/UK-Ham_2002-2003_Met.nc"

metoffice_data_file = open_dataset(metoffice_data_file_address, decode_times = False)

print(metoffice_data_file)

<xarray.Dataset>
Dimensions:    (time: 17520, z: 1, y: 1, x: 1)
Coordinates:
  * time       (time) float64 0.0 1.8e+03 3.6e+03 ... 3.153e+07 3.153e+07
  * z          (z) float64 1.0
  * y          (y) float64 1.0
  * x          (x) float64 1.0
Data variables:
    latitude   (y, x) float64 ...
    longitude  (y, x) float64 ...
    SWdown     (time, y, x) float64 ...
    Tair       (time, y, x) float64 ...
    Precip     (time, y, x) float64 ...
    Qair       (time, y, x) float64 ...
    Wind       (time, y, x) float64 ...
    Psurf      (time, y, x) float64 ...
    LWdown     (time, y, x) float64 ...
    CO2air     (time, z, y, x) float64 ...
    za_tq      (y, x) float64 ...
    za_uv      (y, x) float64 ...
Attributes:
    description:    Alice Holt met data, created by Martin De Kauwe
    history:        Created by: generate_met_file_2022_JULES.py
    creation_date:  2023-06-30 12:33:02.874471
    contact:        mdekauwe@gmail.com


We can take a closer look at the available data "columns" using the keys property.

In [3]:
print(metoffice_data_file.keys)

<bound method Mapping.keys of <xarray.Dataset>
Dimensions:    (time: 17520, z: 1, y: 1, x: 1)
Coordinates:
  * time       (time) float64 0.0 1.8e+03 3.6e+03 ... 3.153e+07 3.153e+07
  * z          (z) float64 1.0
  * y          (y) float64 1.0
  * x          (x) float64 1.0
Data variables:
    latitude   (y, x) float64 ...
    longitude  (y, x) float64 ...
    SWdown     (time, y, x) float64 ...
    Tair       (time, y, x) float64 ...
    Precip     (time, y, x) float64 ...
    Qair       (time, y, x) float64 ...
    Wind       (time, y, x) float64 ...
    Psurf      (time, y, x) float64 ...
    LWdown     (time, y, x) float64 ...
    CO2air     (time, z, y, x) float64 ...
    za_tq      (y, x) float64 ...
    za_uv      (y, x) float64 ...
Attributes:
    description:    Alice Holt met data, created by Martin De Kauwe
    history:        Created by: generate_met_file_2022_JULES.py
    creation_date:  2023-06-30 12:33:02.874471
    contact:        mdekauwe@gmail.com>


Alternativly you can inspect a netcdf file without the need to write python code. For example to view the header of the netcdf file from the terminal use the following command: 

$$ \text{ncdump -h file_address} $$

Note the -h restricts the output to only the header, if it is removed the entier file is printed. 

Alternativly for a graphical interface you can open the file using nasas panoply software (https://www.giss.nasa.gov/tools/panoply/download/).

From this file we only want a subset of the columns, specificaly air temperature, specific humidity, air pressure at the ground surface and the $CO_2$ fraction of the air. These columns are labeld Tair, Qair, Psurf and CO2air respectivly. The code bellow reduces the data to just these columns.

In [4]:
met_columns = ["Tair", "Qair", "Psurf", "CO2air"]
metoffice_data = metoffice_data_file[met_columns]

print(metoffice_data)

<xarray.Dataset>
Dimensions:  (time: 17520, y: 1, x: 1, z: 1)
Coordinates:
  * time     (time) float64 0.0 1.8e+03 3.6e+03 ... 3.153e+07 3.153e+07
  * z        (z) float64 1.0
  * y        (y) float64 1.0
  * x        (x) float64 1.0
Data variables:
    Tair     (time, y, x) float64 ...
    Qair     (time, y, x) float64 ...
    Psurf    (time, y, x) float64 ...
    CO2air   (time, z, y, x) float64 ...
Attributes:
    description:    Alice Holt met data, created by Martin De Kauwe
    history:        Created by: generate_met_file_2022_JULES.py
    creation_date:  2023-06-30 12:33:02.874471
    contact:        mdekauwe@gmail.com


We can see that each column is actualy three dimentional. The first dimension coressponds to the time at wich the measurment is conducted. The remaining two dimentions "x" and "y" are needed when running simulations over a grid of adjacent sites, and "z" is used to brake the atmospher into different cells. The data in this particular file only uses one site and a single hight cell so we can remove these three diemtions.

In [5]:
metoffice_data = metoffice_data.squeeze(dim=["x", "y", "z"], drop = True)

print(metoffice_data)

<xarray.Dataset>
Dimensions:  (time: 17520)
Coordinates:
  * time     (time) float64 0.0 1.8e+03 3.6e+03 ... 3.153e+07 3.153e+07
Data variables:
    Tair     (time) float64 ...
    Qair     (time) float64 ...
    Psurf    (time) float64 ...
    CO2air   (time) float64 ...
Attributes:
    description:    Alice Holt met data, created by Martin De Kauwe
    history:        Created by: generate_met_file_2022_JULES.py
    creation_date:  2023-06-30 12:33:02.874471
    contact:        mdekauwe@gmail.com


Next we convert this into a pandas data frame to make it easier to work with.

In [6]:
metoffice_data = metoffice_data.to_pandas()

print(metoffice_data)

               Tair      Qair     Psurf  CO2air
time                                           
0.0         289.322  0.006778  101608.0  418.56
1800.0      289.322  0.006778  101608.0  418.56
3600.0      287.886  0.006796  101628.0  418.56
5400.0      287.886  0.006796  101628.0  418.56
7200.0      286.193  0.006731  101659.0  418.56
...             ...       ...       ...     ...
31527000.0  285.735  0.006571   99948.7  418.56
31528800.0  286.414  0.006736   99905.7  418.56
31530600.0  286.414  0.006736   99905.7  418.56
31532400.0  285.521  0.007251   99803.2  418.56
31534200.0  285.521  0.007251   99803.2  418.56

[17520 rows x 4 columns]


Inorder to run the profit optimisation model we need the vapour pressure deficit in $kPa$.

In [7]:
metoffice_data['VPD'] = vapour_pressure_deficit(metoffice_data['Tair'],
                                                metoffice_data['Qair'],
                                                metoffice_data['Psurf'])

print(metoffice_data)

               Tair      Qair     Psurf  CO2air       VPD
time                                                     
0.0         289.322  0.006778  101608.0  418.56  0.734297
1800.0      289.322  0.006778  101608.0  418.56  0.734297
3600.0      287.886  0.006796  101628.0  418.56  0.569528
5400.0      287.886  0.006796  101628.0  418.56  0.569528
7200.0      286.193  0.006731  101659.0  418.56  0.405156
...             ...       ...       ...     ...       ...
31527000.0  285.735  0.006571   99948.7  418.56  0.404794
31528800.0  286.414  0.006736   99905.7  418.56  0.445145
31530600.0  286.414  0.006736   99905.7  418.56  0.445145
31532400.0  285.521  0.007251   99803.2  418.56  0.277809
31534200.0  285.521  0.007251   99803.2  418.56  0.277809

[17520 rows x 5 columns]


The time column here is not in the format we necesseraly want so lets create another column with the date and time of each measurement.

In [8]:
units, reference_date = metoffice_data_file.time.attrs['units'].split('since')

start = reference_date.strip()[0:19].replace("-","/")

metoffice_data['date time'] = date_range(start = start, periods = len(metoffice_data), freq = '30MIN')

print(metoffice_data)

               Tair      Qair     Psurf  CO2air       VPD           date time
time                                                                         
0.0         289.322  0.006778  101608.0  418.56  0.734297 2022-01-01 00:00:00
1800.0      289.322  0.006778  101608.0  418.56  0.734297 2022-01-01 00:30:00
3600.0      287.886  0.006796  101628.0  418.56  0.569528 2022-01-01 01:00:00
5400.0      287.886  0.006796  101628.0  418.56  0.569528 2022-01-01 01:30:00
7200.0      286.193  0.006731  101659.0  418.56  0.405156 2022-01-01 02:00:00
...             ...       ...       ...     ...       ...                 ...
31527000.0  285.735  0.006571   99948.7  418.56  0.404794 2022-12-31 21:30:00
31528800.0  286.414  0.006736   99905.7  418.56  0.445145 2022-12-31 22:00:00
31530600.0  286.414  0.006736   99905.7  418.56  0.445145 2022-12-31 22:30:00
31532400.0  285.521  0.007251   99803.2  418.56  0.277809 2022-12-31 23:00:00
31534200.0  285.521  0.007251   99803.2  418.56  0.277809 2022-1